# Entrenamiento y Construcción Modelo Predictivo MVP NBA

Se entrenó un modelo con los datos estadísticos de los jugadores de la NBA desde 1980 hasta 2021

# Obtención de datos

In [1]:
#libreria para conexion a base de datos
import pyodbc 
import pandas as pd
import numpy as np
import warnings #notificaciones
warnings.filterwarnings("ignore")

In [2]:
#conexion a base de datos
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=LAPTOP-HAPH02BA;'
                      'Database=NBA;'
                      'Trusted_Connection=yes;')

In [3]:
#query para obtener datos
query = 'select * from mvp where year < 2022'

In [4]:
#carga de datos a dataframe
df = pd.read_sql_query(query,conn)
df.head()

,Player,Pos,Age,G,year,PTS,AST,STL,TRB,BLK,...,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,mvp,Fecha_act
0,Dwight Howard,PF,19.0,82.0,2005.0,12.0,0.9,0.9,10.0,1.7,...,0.0,0.0,0.000,4.3,8.2,0.521,3.4,5.0,0.0,2022-02-03
1,Josh Howard,SF,24.0,76.0,2005.0,12.6,1.4,1.5,6.4,0.6,...,0.4,1.5,0.296,4.5,8.9,0.506,2.2,3.1,0.0,2022-02-03
2,Juwan Howard,PF,31.0,61.0,2005.0,9.6,1.5,0.5,5.7,0.1,...,0.0,0.0,0.000,4.0,8.9,0.452,1.6,1.9,0.0,2022-02-03
3,Troy Hudson,PG,28.0,79.0,2005.0,8.7,3.6,0.3,1.3,0.1,...,1.1,3.3,0.345,2.2,5.1,0.436,0.9,1.1,0.0,2022-02-03
4,Larry Hughes,SG,26.0,61.0,2005.0,22.0,4.7,2.9,6.3,0.3,...,1.0,3.4,0.282,6.7,14.4,0.465,5.8,7.4,0.0,2022-02-03


# Construcción del modelo

Dada la poca muestra de datos de MVP para cada temporada, solo 1 MVP por temporada, y una cantidad de aproximadamente 300 jugadores por temporada, esto va a llevar a un problema de desbalanceo de datos. Para los fines, enfocamos el esfuerzo en un modelo probabilístico de Naive Bayes, asumiendo que el que tenga la probabilidad más alta será el MVP.

Otro problema es como va cambiando la liga, es decir, el juicio de los expertos va cambiando al igual que el juego durante los años, surgen nuevas estadísticas que hacen que valoren a los jugadores diferente. Por esto el approach que tomamos fue de entrenar el modelo con los años anteriores y probar con cada año, a medida que van pasando los años en el bucle for el modelo irá fortaleciendo y actualizando lo que vaya aprendiendo.

In [5]:
from sklearn.naive_bayes import GaussianNB

years = range(2010,2022)
mvp_years = dict()
resultado_mvp = pd.DataFrame(columns=['Player','Year','MVP'])
nba_mvp_ = df[df['G']>=50]

for y in years:
    X_train = nba_mvp_[nba_mvp_.year < y][['PTS','AST','TRB','FG','WS','PER','VORP','BPM','WS/48']]
    y_train = nba_mvp_[nba_mvp_.year < y][['mvp']]
    X_test = nba_mvp_[nba_mvp_.year == y][['PTS','AST','TRB','FG','WS','PER','VORP','BPM','WS/48']]
    test = nba_mvp_[nba_mvp_.year == y][['Player','mvp']]
    
    gb = GaussianNB()
    gb.fit(X_train,y_train)
    pred_proba = gb.predict_proba(X_test)
    
    y_pred_proba = []
    for i in enumerate(pred_proba):
        y_pred_proba.append(i[1][1])
    
    y_pred_proba = np.asarray(y_pred_proba)
    mvp_years = pd.DataFrame({'Player':test['Player'],
                             'Year':y,
                             'MVP':y_pred_proba,
                             'MVP_Real':test['mvp']})
    resultado_mvp = pd.concat([resultado_mvp,mvp_years], sort=True)

resultado_mvp = resultado_mvp.reset_index(drop=True)

resultado_mvp = resultado_mvp.sort_values(by=['Year','MVP'], ascending=False).groupby('Year').head(1)

resultado_mvp = resultado_mvp.sort_values('Year', ascending=False).drop_duplicates()

# Resultados

In [6]:
#nivel de acierto del modelo
accuracy = round(resultado_mvp['MVP_Real'].sum()/resultado_mvp['MVP'].sum() * 100,1)

print('accuracy: '+ str(accuracy))

accuracy: 75.0


In [7]:
#fallos del modelo
resultado_mvp[resultado_mvp['MVP_Real']==0]

,MVP,MVP_Real,Player,Year
2241,1.0,0.0,James Harden,2017
938,1.0,0.0,Kevin Durant,2013
472,1.0,0.0,LeBron James,2011


Los resultados del modelo son bastantes satisfactorios, principalmente para los años más recientes, evaluando el accuracy de los últimos 11 años, el modelo logró acertar en el 75% de los casos y solo falló en 3 años. Sin embargo los jugadores que el modelo predice que debieron ser MVP, especificamente para el 2011,  2013 y 2017, fueron los 2dos más votados para MVP en las 3 ocasiones y para algunos expertos del deporte, afirman que debieron ser los MVP. Estos son resultados aceptables.

# Salvamiento del modelo

In [8]:
import pickle

modelo_naivebayes_nba_mvp = 'modelo_naivebayes_nba_mvp.sav'
pickle.dump(gb, open(modelo_naivebayes_nba_mvp, 'wb'))